# Extract Sentences from Wikipedia
+ This notebook is used for collecting sentences that tell relationship between two entities from wikipedia using some dependency path pattern
+ **This notebook is fully valid under Owl3 machine (using the /scratch/data/wikipedia/full_text-2021-03-20 data)**

In [1]:
import re
from bs4 import BeautifulSoup
import pandas as pd
import sys
import wikipedia
import os
from wikipedia2vec import Wikipedia2Vec
from collections import Counter
import bz2
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import tqdm

sys.path.append('..')

from tools.BasicUtils import my_read, my_write, my_read_pickle, my_write_pickle
from tools.TextProcessing import (
                my_sentence_tokenize, build_word_tree_v2, 
                my_sentence_tokenize, filter_specific_keywords, find_dependency_path_from_tree, find_span, nlp, 
                exact_match
                )

from extract_wiki import (
    wikipedia_entity_file, record_columns, 
    save_path, entity_occur_file, graph_file, single_sent_graph_file, 
    w2vec_dump_file, w2vec_keyword_file, w2vec_wordtree_file, w2vec_token_file, 
    w2vec_keyword2idx_file, 
    test_path, path_test_file, 
    path_pattern_count_file, 
    save_sub_folders, wiki_sub_folders, 
    wiki_files, save_sent_files, save_cooccur_files, save_selected_files, save_title_files, save_cooccur__files, 
    p, patterns, 
    note2line, line2note, process_file, filter_path_from_df, find_all_triangles, 
    feature_columns, feature_process, gen_pattern, gen_kw_from_wiki_ent, get_entity_page
)

# Generate the save dir
if not os.path.exists(save_path):
    os.mkdir(save_path)

if not os.path.exists(test_path):
    os.mkdir(test_path)

for save_dir in save_sub_folders:
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)

# Get all files under wikipedia/full_text-2021-03-20

print('wiki sub folder example:', wiki_sub_folders[0])
print('save sub folder example:', save_sub_folders[0])
print('wiki file example:', wiki_files[0])
print('save sentence file example:', save_sent_files[0])
print('save cooccur file example:', save_cooccur_files[0])
print('save selected sentence file example:', save_selected_files[0])

wiki sub folder example: ../../data/wikipedia/full_text-2021-03-20/BE
save sub folder example: data/extract_wiki/wiki_sent_collect/BE
wiki file example: ../../data/wikipedia/full_text-2021-03-20/BE/wiki_00
save sentence file example: data/extract_wiki/wiki_sent_collect/BE/wiki_00.dat
save cooccur file example: data/extract_wiki/wiki_sent_collect/BE/wiki_00_co.dat
save selected sentence file example: data/extract_wiki/wiki_sent_collect/BE/wiki_00_se.dat


In [8]:
# [Load] wikipedia2vec
with bz2.open(w2vec_dump_file) as f_in:
    w2vec = Wikipedia2Vec.load(f_in)

/home/keruiz2/miniconda3/envs/FWD_py38/lib/python3.8/contextlib.py:113: UserWarning: "<bz2.BZ2File object at 0x7f4eb0519280>" is not a raw file, mmap_mode "c" flag will be ignored.
  return next(self.gen)


In [ ]:
# [Test] wikipedia2vec

# Find similar words or entities
# ent1 = 'Python (programming language)'
# w2vec.most_similar_by_vector(w2vec.get_entity_vector(ent1), 20)

# Get similarity between two entities
# ent1 = 'Data mining'
# ent2 = 'Database system'
# cosine_similarity(w2vec.get_entity_vector(ent1).reshape(1, -1), w2vec.get_entity_vector(ent2).reshape(1, -1))[0, 0]

# Check the entity count and document count
ent1 = 'Hidden Markov model'
e = w2vec.get_entity(ent1)
print(e.count)
print(e.doc_count)

## [Preparation] Collect sentences, entities, entities co-occurrances, titles from wikipedia dump

In [ ]:
# python extract_wiki.py collect_sent_and_cooccur (8 hours)

In [ ]:
# Collect wikipedia entities
wikipedia_entity = set()
for f in tqdm.tqdm(save_title_files):
    with open(f) as f_in:
        wikipedia_entity.update(f_in.read().split('\n'))
print(len(wikipedia_entity))
my_write(wikipedia_entity_file, list(wikipedia_entity))
        

## [Preparation] Correct entity mapping in co-occurrance files

In [ ]:
wikipedia_entity_low2orig_map_file = 'data/extract_wiki/wikipedia_entity_low2orig_map.pickle'
wikipedia_entity_low2orig_map = {}
with open(wikipedia_entity_file) as f_in:
    wikipedia_entity = set(f_in.read().split('\n'))
    for ent in tqdm.tqdm(wikipedia_entity):
        ent_low = ent.lower()
        if ent_low not in wikipedia_entity_low2orig_map:
            wikipedia_entity_low2orig_map[ent_low] = []
        wikipedia_entity_low2orig_map[ent_low].append(ent)
my_write_pickle(wikipedia_entity_low2orig_map_file, wikipedia_entity_low2orig_map)

In [ ]:
wikipedia_entity_low2orig_map = my_read_pickle(wikipedia_entity_low2orig_map_file)
    
with open(wikipedia_entity_file) as f_in:
    wikipedia_entity = set(f_in.read().split('\n'))
    
for i in tqdm.tqdm(range(len(save_cooccur_files))):
    with open(save_cooccur_files[i]) as f_in:
        new_file_lines = []
        for line_idx, line in enumerate(f_in):
            line = line.strip()
            entities = line.split('\t')
            new_entities = []
            for ent in entities:
                if ent in wikipedia_entity:
                    new_entities.append(ent)
                else:
                    ent_low = ent.lower()
                    if ent_low in wikipedia_entity_low2orig_map:
                        candidates = wikipedia_entity_low2orig_map[ent_low]
                        if len(candidates) == 1:
                            new_entities.append(candidates[0])
                        else:
                            note = line2note(save_cooccur_files[i], line_idx, '_co.dat')
                            page_title = note2line(note, '_ti.dat').strip()
                            try:
                                page_ent_vec = w2vec.get_entity_vector(page_title)
                            except:
                                continue
                            most_similar_idx, most_similar_val = -1, -1
                            for candidate_idx, candidate_ent in enumerate(candidates):
                                try:
                                    candidate_vec = w2vec.get_entity_vector(candidate_ent)
                                except:
                                    continue
                                similar_val = cosine_similarity(page_ent_vec.reshape(1, -1), candidate_vec.reshape(1, -1))[0,0]
                                if similar_val > most_similar_val:
                                    most_similar_val = similar_val
                                    most_similar_idx = candidate_idx
                            if most_similar_idx >= 0:
                                new_entities.append(candidates[most_similar_idx])
            new_file_lines.append('\t'.join(new_entities))
        my_write(save_cooccur__files[i], new_file_lines)

In [ ]:
# [Test]
lines = get_entity_page('Machine learning')
sents = [note2line(note) for note in lines]
occurs = [note2line(note, '_co_.dat') for note in lines]
ori_occurs = [note2line(note, '_co.dat') for note in lines]
my_write('sent_check.txt', sents)
my_write('occur_check.txt', occurs)
my_write('ori_occur_check.txt', ori_occurs)
lines[0]

## [Preparation] Mapping keyword mention to wikipedia2vec entities

In [ ]:
with open(wikipedia_entity_file) as f_in:
    wikipedia_entity = set(f_in.read().split('\n'))
    
w2vec_keyword2idx = {}

for entity in tqdm.tqdm(wikipedia_entity):
    w2vec_entity = w2vec.get_entity(entity)
    if w2vec_entity is None:
        continue
    kw = gen_kw_from_wiki_ent(entity)
    if kw not in w2vec_keyword2idx:
        w2vec_keyword2idx[kw] = [w2vec_entity.index]
    else:
        if w2vec_entity.index not in w2vec_keyword2idx[kw]:
            w2vec_keyword2idx[kw].append(w2vec_entity.index)
w2vec_kws = filter_specific_keywords(list(w2vec_keyword2idx.keys()))
my_write(w2vec_keyword_file, w2vec_kws)
build_word_tree_v2(w2vec_keyword_file, w2vec_wordtree_file, w2vec_token_file)
filter_keyword_from_w2vec = set(w2vec_kws)
w2vec_keyword2idx = {k:v for k, v in w2vec_keyword2idx.items() if k in filter_keyword_from_w2vec}
my_write_pickle(w2vec_keyword2idx_file, w2vec_keyword2idx)
len(w2vec_keyword2idx)

In [ ]:
# [Load] w2vec_keyword2idx
w2vec_keyword2idx = my_read_pickle(w2vec_keyword2idx_file)

In [ ]:
# [Test] w2vec_keyword2idx
kw = 'feature engineering'
kw_in_mention = kw in w2vec_keyword2idx
print(kw_in_mention)
if kw_in_mention:
    for idx in w2vec_keyword2idx[kw]:
        print(w2vec.dictionary.get_item_by_index(idx))

## [Preparation] Collect dataset

In [ ]:
# [Create] Collect sample data using general wikipedia2vec keywords and wiki sent files
wiki_path_test_df = pd.concat([pd.DataFrame(process_file(save_sent_files[file_idx], save_cooccur__files[file_idx], w2vec)) for file_idx in range(8)], ignore_index=True)
wiki_path_test_df.to_csv(path_test_file, sep='\t', columns=feature_columns, index=False)
print(len(wiki_path_test_df))

In [ ]:
# [Load] Load path test data (pd.DataFrame)
wiki_path_test_df = pd.read_csv(open(path_test_file), sep='\t')

In [ ]:
# [Create] Pattern frequency generation

sub_df = wiki_path_test_df[wiki_path_test_df['sim'] > 0.5]

sub_df = sub_df.assign(pick = sub_df.apply(lambda x: 1 if 'nsubj' in x['dep_path'] else 0, axis=1))
sub_df = sub_df[sub_df['pick'] > 0]

sub_df['pattern'] = sub_df.apply(lambda x: gen_pattern(x['dep_path']), axis=1)

c = Counter(sub_df['pattern'].to_list())

my_write_pickle(path_pattern_count_file, c)

In [ ]:
c.most_common(10)

In [ ]:
# [Load] cal_freq function

c = my_read_pickle(path_pattern_count_file)

max_cnt = c.most_common(1)[0][1]
log_max_cnt = np.log(max_cnt+1)

def cal_freq(path:str):
    cnt = c.get(path)
    cnt = (cnt if cnt else 0.5) + 1
    return np.log(cnt) / log_max_cnt

In [ ]:
# [Test] cal_freq function
cal_freq('i_nsubj prep pobj prep pobj')

In [ ]:
# [Test] Collect sample data using general wikipedia2vec keywords and wiki sent files
sub_df = filter_path_from_df(wiki_path_test_df, cal_freq)
sub_df = sub_df.assign(sent = sub_df.apply(lambda x: note2line(x['sent']).strip(), axis=1))
sub_df.to_csv('full_phrase_check.tsv', columns = ['sent', 'kw1', 'kw1_recall', 'kw1_full_span', 'kw2', 'kw2_recall', 'kw2_full_span'], sep='\t', index=False)

In [ ]:
# [Create] collect dataset [collect_dataset]

## [Prepration] Generate Graph

In [ ]:
# Generate the graph ['generate_graph']

## Examine Dataset

In [ ]:
# Get the entity occurance ['collect_ent_occur_from_selected']

In [2]:
# [Load] entity occur dict which has occurance record for all entities in selected sentences
entity_occur = my_read_pickle(entity_occur_file)

In [ ]:
# Demo check co-occur of two entities in selected sentences
def get_selected_record(entity_dict:dict, ent1:str, ent2:str):
    kw1_occur = entity_dict.get(ent1)
    kw2_occur = entity_dict.get(ent2)
    if not kw1_occur or not kw2_occur:
        return None
    co_occur = kw1_occur & kw2_occur
    data = []
    for occur in co_occur:
        record = note2line(occur, '_se.dat').strip().split('\t')
        sent = note2line(record[7]).strip()
        data_dict = {record_columns[i] : record[i] for i in range(len(record))}
        data_dict['sent'] = sent
        data.append(data_dict)
    
    df = pd.DataFrame(data = data)
    return df

In [ ]:
# [Test] entity occur dict

ent1 = 'Artificial intelligence'
# Check the existance of an entity
ent1 in entity_occur

# Check the sentences where an entity appear
# for note in entity_occur[ent1]:
#     print(note2line(note2line(note, '_se.dat').split('\t')[7]).strip())

# Check the records of two entities
ent2 = 'Natural language processing'
df = get_selected_record(entity_occur, ent1, ent2)
if df is not None:
    print(len(df))
    df.to_csv('sents.tsv', columns=['score'] + feature_columns + ['pattern', 'pattern_freq'], sep='\t', index=False)

In [ ]:
# [Load] Graph
graph = my_read_pickle(graph_file)

print('num of nodes:', len(graph.nodes))
print('num of edges:', len(graph.edges))

In [ ]:
# [Test] graph
ent1 = 'Data mining'
# Check the neighbours of an entity
# list(graph.neighbors(ent1))

# Check the edges of two entities
graph.edges[ent1, 'Data fusion']

In [ ]:
# Show sentence from file
note2line('AC:32:2390')

In [ ]:
node2neig_cnt = {node : len(list(graph.neighbors(node))) for node in graph.nodes.keys()}

In [ ]:
neig_cnt = [v for v in node2neig_cnt.values() if v < 20]
plt.title('num of nodes vs num of neighbors each node')
plt.hist(neig_cnt)
plt.show()

In [ ]:
node2triangle_num = nx.triangles(graph)
my_write_pickle('node2tri_num.pickle', node2triangle_num)

In [ ]:
print('num of triangles:', sum(node2triangle_num.values()) / 3)
plt.title('num of nodes vs num of triangles each node')
plt.hist([v for v in node2triangle_num.values() if v >= 1 and v < 10])
plt.show()

## Generate training data from triangles

In [ ]:
# [Load] Single sentence graph
single_sent_graph = my_read_pickle(single_sent_graph_file)
edges = [edge for edge in tqdm.tqdm(single_sent_graph.edges) if single_sent_graph.get_edge_data(*edge)['score'] > 0.65]
filtered_graph = single_sent_graph.edge_subgraph(edges)
nodes = []
for node in tqdm.tqdm(filtered_graph):
    ent = w2vec.get_entity(node)
    if ent is None:
        continue
    if ent.count >= 20:
        nodes.append(node)
filtered_graph = filtered_graph.subgraph(nodes)
print('number of nodes:', filtered_graph.number_of_nodes())
print('number of edges:', filtered_graph.number_of_edges())

In [21]:
from typing import List
from nltk.corpus import stopwords
self_define_stopwords = set(['-', ',', '.'])
sw = set(stopwords.words('english'))
import math

def find_triangles(graph:nx.Graph, node:str):
    triangles = set()
    neighbors = set(graph.neighbors(node))
    for neighbor in neighbors:
        second_neighbors = set(graph.neighbors(neighbor))
        inter_neighbors = neighbors & second_neighbors
        for third_neighbor in inter_neighbors:
            triangles.add((node, neighbor, third_neighbor) if neighbor < third_neighbor else (node, third_neighbor, neighbor))
    return triangles


def find_triangle_with_node(graph:nx.Graph, first_node:str, second_node:str='', third_node:str=''):
    triangles = list(find_triangles(graph, first_node))
    triangles.sort(key=lambda x: x[1])
    triangle_with_sents = []
    n_seen = set()
    for n1, n2, n3 in triangles:
        if second_node and n2 != second_node and n3 != second_node:
            continue
        if third_node and n2 != third_node and n3 != third_node:
            continue
        if n2 not in n_seen:
            n_seen.add(n2)
            triangle_with_sents.append((n1, note2line(graph.get_edge_data(n1, n2)['note']).strip(), n2, graph.get_edge_data(n1, n2)['score']))
        if n3 not in n_seen:
            n_seen.add(n3)
            triangle_with_sents.append((n1, note2line(graph.get_edge_data(n1, n3)['note']).strip(), n3, graph.get_edge_data(n1, n3)['score']))
        triangle_with_sents.append((n2, note2line(graph.get_edge_data(n3, n2)['note']).strip(), n3, graph.get_edge_data(n3, n2)['score']))
    return triangle_with_sents


def isf(w:str, D:int, counters:List[Counter]):
    return math.log(D * 1.0 / sum([1 if w in sent else 0 for sent in counters]))


def do_pagerank(sents:List[str]):
    # Remove stop words
    clean_sents = [[token for token in sent.split() if token not in sw and token not in self_define_stopwords] for sent in sents]

    # Generate word counters
    counters = [Counter(sent) for sent in clean_sents]

    # Build similarity matrix
    D = len(clean_sents)
    sim_matrix = np.zeros((D, D))
    part_list = [math.sqrt(sum([(sent[w] * isf(w, D, counters)) ** 2 for w in sent])) for sent in counters]
    # return part_list
    for i in range(D - 1):
        for j in range(i + 1, D):
            sent_1 = counters[i]
            sent_2 = counters[j]
            share_word_set = sent_1 & sent_2
            numerator = sum([(sent_1[w] * sent_2[w] * (isf(w, D, counters) ** 2)) for w in share_word_set])
            denominator = part_list[i] * part_list[j]
            sim_matrix[i, j] = numerator / denominator
    sim_matrix = sim_matrix + sim_matrix.T
    g = nx.from_numpy_array(sim_matrix)
    score = nx.pagerank(g)
    temp = sorted(score.items(), key=lambda x: x[1], reverse=True)
    idx = [item[0] for item in temp]
    return [sents[i] for i in idx], [score[i] for i in idx]

In [28]:
test_triangles = find_triangle_with_node(filtered_graph, 'Machine learning', 'Artificial neural network', 'Deep learning')
test_triangles

[('Machine learning',
  'artificial neural network ( ann ) is a subfield of the research area machine learning .',
  'Artificial neural network',
  0.846597576357714),
 ('Machine learning',
  'deep learning ( also known as deep structured learning ) is part of a broader family of machine learning method based on artificial neural network with representation learning .',
  'Deep learning',
  0.7140114888712812),
 ('Artificial neural network',
  'deep learning consists of multiple hidden layer in an artificial neural network .',
  'Deep learning',
  0.8960311100255893)]

In [29]:
sent_list = [triangle[1] for triangle in test_triangles]
sents, score = do_pagerank(sent_list)
list(zip(score, sents))

[(0.48648582432442083,
  'deep learning ( also known as deep structured learning ) is part of a broader family of machine learning method based on artificial neural network with representation learning .'),
 (0.29493351239384963,
  'artificial neural network ( ann ) is a subfield of the research area machine learning .'),
 (0.21858066328172915,
  'deep learning consists of multiple hidden layer in an artificial neural network .')]

## Demo

In [ ]:
# Analyze sentence
doc = nlp('sephardi were exempt from the ban , but it appears that few applied for a letter of free passage .')

# Check noun phrases in the sentences
print(list(doc.noun_chunks))

In [ ]:
len(doc)

In [ ]:
doc = nlp('ada is a structured , statically typed , imperative , and object-oriented high-level programming language , extended from pascal and other language .')
pairs = [{'kw1' : 'ada', 'kw2' : 'programming language'}]
feature_process(doc, pairs)

## Online operations

In [ ]:
def collect_sents_from_wiki_page(page:wikipedia.WikipediaPage):
    remove_list = ['See also', 'References', 'Further reading', 'Sources', 'External links']
    dic = {sec : page.section(sec) for sec in page.sections}
    dic['summary'] = page.summary
    sents = []
    section_list = list(dic.keys())
    while len(section_list) > 0:
        section = section_list.pop()
        if section in remove_list:
            continue
        section_text = dic[section]
        if not section_text:
            continue
        # processed_text = clean_text(section_text)
        processed_text = ' '.join(section_text.lower().split())
        temp_sents = my_sentence_tokenize(processed_text, True)
        sents += temp_sents
    return list(sents)

def collect_entity_from_wiki_page(page:wikipedia.WikipediaPage):
    return [text.lower() for text in page.links]

def collect_keyword_from_wiki_page(page:wikipedia.WikipediaPage):
    soup = BeautifulSoup(page.html(), 'html.parser')
    main_block = soup.find('div', class_='mw-parser-output')
    keywords = set([l.text.lower() for l in main_block.findAll('a') if re.match(r'^(<a href="/wiki/)', str(l))])
    return keywords



In [ ]:
keyword = 'python'

p = wikipedia.page(keyword)
if p is not None:
    sents = collect_sents_from_wiki_page(p)
    keywords = collect_keyword_from_wiki_page(p)
    print('sentences collected')
    my_write('%s.txt' % keyword, sents)
    my_write('%s_kw.txt' % keyword, keywords)
    df = filter_by_path(sents)
    df.to_csv('%s_out.tsv' % keyword, sep='\t', index=False)

    dff = df[df.apply(lambda x: str(x['head']) in keywords and str(x['tail']) in keywords, axis=1)]
    dff.to_csv('%s_out_f.tsv' % keyword, sep='\t', index=False)

# Appendix

## Hand-crafted analysis

In [ ]:
wiki_test_df = wiki_path_test_df[wiki_path_test_df['sim'] >= 0.0]

def match_path_pattern(path:str):
    for pp in patterns:
        if exact_match(pp, path):
            return pp
    return ''

wiki_test_df['pattern'] = wiki_test_df.apply(lambda x: match_path_pattern(x['path']), axis=1)

In [ ]:
def analysis_path_result_sim_based(df:pd.DataFrame, paths:list):
    summary_df = pd.DataFrame(columns=['path', 'cnt', 'ratio', 'avg_sim'])
    for pp in paths:
        sub_df = df[df['pattern'] == pp]
        summary_df = summary_df.append({
            'path' : pp,
            'cnt' : len(sub_df),
            'ratio' : len(sub_df) / len(df),
            'avg_sim' : sum(sub_df['sim']) / len(sub_df) if len(sub_df) else 0
        }, ignore_index=True)
    summary_df = summary_df.append({
        'path' : 'general',
        'cnt' : len(df),
        'ratio' : 1,
        'avg_sim' : sum(df['sim']) / len(df) if len(df) else 0
    }, ignore_index=True)
    return summary_df

In [ ]:
analysis_path_result_sim_based(wiki_test_df, patterns)

In [ ]:
def collect_example_sent_for_pattern(df:pd.DataFrame, path:str, num:int=30, posfix:str='.dat'):
    sub_df = df[df['pattern'] == path]
    num = min(len(sub_df), num)
    sub_df = sub_df[:num]
    sub_df['sent'] = sub_df.apply(lambda x: note2line(x['sent'], posfix=posfix).strip(), axis=1)
    return sub_df

for patt in patterns:
    temp_df = collect_example_sent_for_pattern(wiki_test_df, patt)
    temp_df.to_csv('%s.tsv' % (patt[:10] if len(patt) >= 10 else patt), index=False, sep='\t')

In [33]:
triangle_set = my_read_pickle('data/extract_wiki/triangles.pickle')

In [32]:
len(triangle_set)

818486

In [34]:
for i, tri in enumerate(triangle_set):
    print(tri)
    if i > 10:
        break

frozenset({'Kennebec County, Maine', 'Waterville, Maine', 'Maine'})
frozenset({'Inorganic compound', 'Lithium carbonate', 'Sodium carbonate'})
frozenset({'Sixth form', "Alleyn's School", 'Church of England'})
frozenset({'Illinois', 'Texas', 'Ohio'})
frozenset({'Star', 'Exoplanet', 'Substellar object'})
frozenset({'John Scofield', 'Michel Jonasz', 'Michel Portal'})
frozenset({'Montreal', 'Quebec', 'La Tuque, Quebec'})
frozenset({'Star', 'Solar radius', 'Sun'})
frozenset({'Plagioclase', 'Olivine', 'Quartz'})
frozenset({'First Army (Bulgaria)', 'Debar', 'Struga'})
frozenset({'Oak Ridge, Tennessee', 'Knoxville, Tennessee', 'Roane County, Tennessee'})
frozenset({'Transcaucasia', 'North Africa', 'Middle East'})


In [35]:
samples = []
for i, tri in enumerate(triangle_set):
    samples.append(find_triangle_with_node(single_sent_graph, *tri))
    if i > 10:
        break


In [36]:
samples

[[('Kennebec County, Maine',
   'kennebec county is a county in the u.s. state of maine , in the united state .',
   'Maine',
   0.7699937343936527),
  ('Kennebec County, Maine',
   'waterville is located in northern kennebec county at .',
   'Waterville, Maine',
   0.8214094636421466),
  ('Maine',
   'maine central operated parallel main line between royal junction and waterville .',
   'Waterville, Maine',
   0.7669628446642484)],
 [('Inorganic compound',
   'lithium carbonate is an inorganic compound , the lithium salt of carbonate with the chemical formula .',
   'Lithium carbonate',
   0.6549696543360262),
  ('Inorganic compound',
   'sodium carbonate , , ( also known as washing soda , soda ash and soda crystal ) is the inorganic compound with the formula naco and its various hydrate .',
   'Sodium carbonate',
   0.7255028023549482),
  ('Lithium carbonate',
   'unlike sodium carbonate , which forms at least three hydrate , lithium carbonate exists only in the anhydrous form .',
  